In [1]:
import sys
import laspy
import requests
import wget
import os
print(os.getcwd())
arcpy.env.addOutputsToMap = False

C:\Windows\System32


In [2]:
# sciezki
# folder z pobranymi lidarami
lidars_path = r"C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\dane"
lidars_names = os.listdir(lidars_path)

wyniki = r"C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki"

print(lidars_names)

['76632_1261787_M-34-5-D-b-2-3-2.laz', '76632_1261789_M-34-5-D-b-2-3-4.laz', '76632_1261790_M-34-5-D-b-2-4-1.laz', '76632_1261791_M-34-5-D-b-2-4-2.laz', '76632_1261792_M-34-5-D-b-2-4-3.laz', '76632_1261793_M-34-5-D-b-2-4-4.laz']


In [9]:
# pobieranie za pomoca pliku wykonywalnego, środowisko arcgis pro nie wspiera biblioteki wget

#import subprocess
#subprocess.run([r"C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\laz_download.exe", "https://opendata.geoportal.gov.pl/NumDaneWys/DanePomiaroweLAZ/76632/76632_1261787_M-34-5-D-b-2-3-2.laz"])

CompletedProcess(args=['C:\\Users\\janek\\Documents\\ArcGIS\\Projects\\widzenie_maszynowe_laz\\main.exe', 'https://opendata.geoportal.gov.pl/NumDaneWys/DanePomiaroweLAZ/76632/76632_1261787_M-34-5-D-b-2-3-2.laz'], returncode=0)

In [3]:
def convert_laz_to_las(path, wyniki, i):
    output = wyniki + "\\" + "output_" + str(i) + ".lasd"
    
    #arcpy.conversion.ConvertLas(r"C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\dane\76632_1261787_M-34-5-D-b-2-3-2.laz", r"C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki", "SAME_AS_INPUT", '', "NO_COMPRESSION", None, r"C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\output.lasd", "NO_FILES", None)
    arcpy.conversion.ConvertLas(path, wyniki, "SAME_AS_INPUT", '', "NO_COMPRESSION", None, output , "NO_FILES", None)
    

In [4]:

for i,file in enumerate(lidars_names):
    path = lidars_path + "\\" + file
    print(path)
    
    convert_laz_to_las(path, wyniki, i)



C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\dane\76632_1261787_M-34-5-D-b-2-3-2.laz
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\dane\76632_1261789_M-34-5-D-b-2-3-4.laz
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\dane\76632_1261790_M-34-5-D-b-2-4-1.laz
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\dane\76632_1261791_M-34-5-D-b-2-4-2.laz
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\dane\76632_1261792_M-34-5-D-b-2-4-3.laz
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\dane\76632_1261793_M-34-5-D-b-2-4-4.laz


In [5]:
# wybranie tylko las'ow, bo sie tez tworza lasx

list_of_las = []
list_of_files = os.listdir(wyniki)
for f in list_of_files:
    if f[-3:] == "las":
        list_of_las.append(wyniki +  "\\" + f)
        print(wyniki +  "\\" + f)





C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\76632_1261787_M-34-5-D-b-2-3-2.las
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\76632_1261789_M-34-5-D-b-2-3-4.las
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\76632_1261790_M-34-5-D-b-2-4-1.las
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\76632_1261791_M-34-5-D-b-2-4-2.las
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\76632_1261792_M-34-5-D-b-2-4-3.las
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\76632_1261793_M-34-5-D-b-2-4-4.las


In [6]:
# selekcja budynkow

import laspy
import numpy as np



def point_extraction_based_on_the_class(las, clas):
    if clas == 'buildings':
        print('Buildings extraction')
        buildings_only = np.where(las.raw_classification == 6)
        buildings_points = las.points[buildings_only] 

        #print(type(buildings_points))
        #print(buildings_points)

        return buildings_points 
    elif clas == 'vegetation':
        print('Vegetation extraction')
        vegetation_low = np.where(las.raw_classification == 3)
        vegetation_medium = np.where(las.raw_classification == 4)
        vegetation_high = np.where(las.raw_classification == 5)
        vegetation = np.concatenate((vegetation_low,vegetation_medium, vegetation_high))
        vegetation = las.points([vegetation])
        return vegetation
    else:
        print('Ground extraction')
        ground_only = np.where(las.raw_classification == 2)
        ground_points = las.points[ground_only]
        return ground_points 





def save_points_after_processing(file, las, new_las):
    las_pcd = laspy.file.File(file, header=las.header , mode = "w")
    las_pcd.points = new_las #i.e. ground_points
    las_pcd.close()
    
for i, file in enumerate(list_of_las):
    las_pcd = laspy.file.File(file, header =None, mode = "r")
    new_las = point_extraction_based_on_the_class(las_pcd, 'buildings')
    
    
    save_path = wyniki + "\\" + "budynki_" + str(i)+ ".las"
    print(save_path)
    save_points_after_processing(save_path , las_pcd, new_las)


Buildings extraction
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\budynki_0.las
Buildings extraction
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\budynki_1.las
Buildings extraction
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\budynki_2.las
Buildings extraction
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\budynki_3.las
Buildings extraction
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\budynki_4.las
Buildings extraction
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\budynki_5.las


In [7]:
# usuwanie las'ow i lasx'ow
list_of_files = os.listdir(wyniki)
for f in list_of_files:
    if f[0:7] !="budynki":
        print(wyniki + "\\" +f)
        
        
        # czasami lockuje pliki i ich nie chce usunac, arcgis pro...
        try:
            arcpy.management.Delete(wyniki + "\\" +f)
        except:
            pass
        

C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\76632_1261787_M-34-5-D-b-2-3-2.las
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\76632_1261787_M-34-5-D-b-2-3-2.lasx
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\76632_1261789_M-34-5-D-b-2-3-4.las
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\76632_1261789_M-34-5-D-b-2-3-4.lasx
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\76632_1261790_M-34-5-D-b-2-4-1.las
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\76632_1261790_M-34-5-D-b-2-4-1.lasx
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\76632_1261791_M-34-5-D-b-2-4-2.las
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\76632_1261791_M-34-5-D-b-2-4-2.lasx
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\76632_1261792_M-34-5-D-b-2-4-3.las
C:\Users\janek\Documents\ArcGIS\Projects\w

In [8]:
# selekcja budynkow

list_of_buildings = []
list_of_files = os.listdir(wyniki)
for f in list_of_files:

    if f[0:7] == "budynki":
        print(wyniki + "\\" + f)
        list_of_buildings.append(wyniki + "\\" + f)


C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\budynki_0.las
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\budynki_1.las
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\budynki_2.las
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\budynki_3.las
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\budynki_4.las
C:\Users\janek\Documents\ArcGIS\Projects\widzenie_maszynowe_laz\wyniki\budynki_5.las


In [9]:
#budynki las na shp

shps = []

for i, b in enumerate(list_of_buildings):
    shp = wyniki + "\\" + "punkty_las_" +str(i)
    
    shps.append(shp)
    
    arcpy.ddd.LASToMultipoint(
    input=b,
    out_feature_class=shp,
    average_point_spacing=1,
    class_code=[],
    attribute=None,
    input_coordinate_system='PROJCS["ETRS_1989_Poland_CS92",GEOGCS["GCS_ETRS_1989",DATUM["D_ETRS_1989",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",-5300000.0],PARAMETER["Central_Meridian",19.0],PARAMETER["Scale_Factor",0.9993],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]',
    file_suffix="las",
    z_factor=1,
    folder_recursion="NO_RECURSION"
                            )
print(shps)



['C:\\Users\\janek\\Documents\\ArcGIS\\Projects\\widzenie_maszynowe_laz\\wyniki\\punkty_las_0', 'C:\\Users\\janek\\Documents\\ArcGIS\\Projects\\widzenie_maszynowe_laz\\wyniki\\punkty_las_1', 'C:\\Users\\janek\\Documents\\ArcGIS\\Projects\\widzenie_maszynowe_laz\\wyniki\\punkty_las_2', 'C:\\Users\\janek\\Documents\\ArcGIS\\Projects\\widzenie_maszynowe_laz\\wyniki\\punkty_las_3', 'C:\\Users\\janek\\Documents\\ArcGIS\\Projects\\widzenie_maszynowe_laz\\wyniki\\punkty_las_4', 'C:\\Users\\janek\\Documents\\ArcGIS\\Projects\\widzenie_maszynowe_laz\\wyniki\\punkty_las_5']


In [10]:
# usuwanie budynkow

list_of_files = os.listdir(wyniki)
for f in list_of_files:
    if f[-3:] =="las":
        #print(wyniki + "\\" +f)
        print(f)
        
        
        
        # czasami lockuje pliki i ich nie chce usunac, arcgis pro...
        try:
            arcpy.management.Delete(wyniki + "\\" +f)
        except:
            pass
        




76632_1261793_M-34-5-D-b-2-4-4.las
budynki_0.las
budynki_1.las
budynki_2.las
budynki_3.las
budynki_4.las
budynki_5.las


In [11]:
# pkt na poligony

for i, s in enumerate(shps):
    arcpy.cartography.AggregatePoints(
    in_features=s,
    out_feature_class=wyniki + "\\" + "punkty_las_AggregatePoints_"+str(i),
    aggregation_distance="4 Meters"
                    )
    
    



In [12]:
# czysczenie punktow
list_of_files = os.listdir(wyniki)
for f in list_of_files:
    if "AggregatePoints" not in f:
        # czasami lockuje pliki i ich nie chce usunac, arcgis pro...
        try:
            arcpy.management.Delete(wyniki + "\\" +f)
        except:
            pass
        
